# College Basketball Code
## Dan McDonough
## February 25th, 2021

The purpose of this notebook is to compile four factors stats for D1 NCAAM teams ahead of march madness. My goal is to write functionality that gives a very condensed, stylistic game preview to digest ahead of watching the game.

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import random
import timeit
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import scipy as sp

In [2]:
combined_df = pd.read_csv('college.csv')

combined_df['FTR'] = combined_df['FTR'] * 100
combined_df['EFG'] = combined_df['EFG'] * 100
combined_df['Opp FTR'] = combined_df['Opp FTR'] * 100
combined_df['Opp EFG'] = combined_df['Opp EFG'] * 100

### 2. Write Summary Function

In [3]:
def game_summary(away,home):
    away_df = combined_df[combined_df['Team']==away]
    home_df = combined_df[combined_df['Team']==home]
    home_df_adj = home_df[['Team', 'SRS', 'SOS', 'Pace','DEF', 'Opp FTR', 'Opp EFG', 'Opp TOV', 'Opp ORB', 'OFF', 'FTR', 
                           'EFG', 'TOV', 'ORB']].copy()
    away_df.columns = ['Team', 'SRS', 'SOS', 'Pace','Away Rtg', 'Away FTR', 'Away EFG', 'Away TOV', 'Away ORB',
                       'Home Rtg', 'Home FTR', 'Home EFG', 'Home TOV', 'Home ORB']
    home_df_adj.columns = ['Team', 'SRS', 'SOS', 'Pace','Away Rtg', 'Away FTR', 'Away EFG', 'Away TOV', 'Away ORB',
                       'Home Rtg', 'Home FTR', 'Home EFG', 'Home TOV', 'Home ORB']
    output_df = pd.concat([away_df,home_df_adj],axis=0)
    means_df = pd.DataFrame(output_df.mean()).transpose()
    output_df = pd.concat([output_df,means_df],sort=False).reset_index(drop=True)
    output_df.loc[2,'Team']='Average'
    for column in output_df.columns[1:]:
        output_df[column] = output_df[column].astype(int)
    return output_df[['Team', 'SRS', 'SOS', 'Pace','Away Rtg', 'Away EFG', 'Away TOV', 'Away ORB', 'Away FTR',
                       'Home Rtg', 'Home EFG', 'Home TOV', 'Home ORB','Home FTR']]

Thursday

In [4]:
game_summary('Michigan','Colorado State')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Michigan,14,10,67,108,52,14,31,28,103,50,13,23,26
1,Colorado State,12,4,65,99,49,16,24,24,111,55,13,22,30
2,Average,13,7,66,103,51,15,28,26,107,52,13,22,28


Pregame: Pass at +2.5

Away: Rebounding

Home: Shooting

In [5]:
game_summary('South Dakota State','Providence')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,South Dakota State,7,-3,71,120,60,13,24,33,101,51,14,23,20
1,Providence,12,7,66,99,46,13,28,26,106,50,15,30,38
2,Average,9,1,68,109,53,13,26,30,104,50,14,26,29


Pregame: Lean Providence at -2 

Away: Shooting, free throws

Home: Rebounding, free throws

In [6]:
game_summary('Memphis','Boise State')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Memphis,14,7,71,105,52,19,37,38,95,46,17,31,32
1,Boise State,12,5,65,92,48,18,21,25,104,51,16,30,35
2,Average,13,6,68,99,50,18,29,32,100,49,16,30,33


Pregame: Pass at +2.5

Away: Rebounding and free throws

Home: Shots

In [7]:
game_summary('Norfolk State','Baylor')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Norfolk State,-3,-10,71,105,52,17,30,37,89,43,17,26,32
1,Baylor,21,9,68,92,47,20,28,26,111,52,15,36,28
2,Average,9,0,69,99,50,18,29,32,100,48,16,31,30


Pregame: Pass at -20.5

Away: Shooting, Free throws

Home: Shooting, Rebounding

In [8]:
game_summary('Longwood','Tennessee')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Longwood,0,-7,67,111,52,14,34,34,95,49,18,23,25
1,Tennessee,20,10,68,90,46,20,27,31,105,50,15,32,29
2,Average,10,1,68,101,49,17,31,33,100,49,17,28,27


Pregame: Pass at -17.5

Away: Shooting, turnovers, rebounding

Home: Rebounding

In [9]:
game_summary('Richmond','Iowa')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Richmond,7,3,67,105,51,13,22,30,100,50,16,27,23
1,Iowa,19,7,70,100,49,16,30,26,118,53,11,32,30
2,Average,13,5,68,103,50,14,26,28,109,52,13,29,26


Pregame: Pass at -10.5

Away: Rebounding

Home: Turnovers, Rebounding, Free Throws

In [10]:
game_summary('Georgia State','Gonzaga')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Georgia State,-1,-2,68,102,46,14,34,29,93,48,20,30,30
1,Gonzaga,26,4,73,88,43,15,23,22,119,59,14,29,29
2,Average,12,0,71,95,44,14,28,25,106,53,17,29,30


Pregame: Pass at -23.5

Away: Rebounding, Free Throws

Home: Shooting, Turnovers

In [11]:
game_summary('New Mexico State','Connecticut')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,New Mexico State,6,0,68,106,52,17,33,34,94,45,15,25,27
1,Connecticut,17,7,67,95,45,15,27,31,109,49,14,37,30
2,Average,11,3,67,100,49,16,30,33,102,47,14,31,28


Pregame: Lean UConn at -7

Away: Shooting, Rebounding

Home: Rebounding

In [12]:
game_summary("Saint Peter's",'Kentucky')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Saint Peter's,0,-3,66,99,48,17,32,37,91,43,18,27,40
1,Kentucky,21,8,69,94,46,15,24,25,114,53,14,37,27
2,Average,11,2,68,97,47,16,28,31,103,48,16,32,33


Pregame: Pass at -18

Away: Rebounding, Free Throws

Home: Shooting, Rebounding, Free Throws

In [13]:
game_summary("Indiana","Saint Mary's (CA)")

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Indiana,13,8,67,103,51,15,27,33,95,45,15,24,30
1,Saint Mary's (CA),14,6,65,92,47,17,22,24,106,52,15,27,23
2,Average,14,7,66,98,49,16,24,28,101,49,15,26,27


Pregame: Pass at -2

Away: Rebounding, Free Throws

Home: Shooting, Free Throws

In [14]:
game_summary('Creighton','San Diego State')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Creighton,11,8,68,100,50,18,28,26,96,44,13,28,20
1,San Diego State,13,6,65,87,44,19,27,30,99,48,16,29,31
2,Average,12,7,66,94,47,18,27,28,97,46,15,29,25


Pregame: Pass at -2.5

Away: Shooting

Home: Free Throws

In [15]:
game_summary('Vermont','Arkansas')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Vermont,7,-6,65,114,57,13,24,26,92,47,15,18,22
1,Arkansas,16,8,72,94,47,18,25,32,106,49,15,30,37
2,Average,12,0,68,104,52,15,25,29,99,48,15,24,30


Pregame: Lean Arkansas at -5

Away: Shooting, Turnovers, Free Throws

Home: Rebounding, Free Throws

In [16]:
game_summary('San Francisco','Murray State')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,San Francisco,14,5,70,108,53,15,30,28,94,46,16,23,32
1,Murray State,10,-3,68,91,47,18,24,25,116,54,14,36,31
2,Average,12,0,69,99,50,17,27,27,105,50,15,29,31


Pregame: Lean SF at -1.5

Away: Shooting, Rebounding

Home: Shooting, Rebounding

In [17]:
game_summary('Akron','UCLA')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Akron,0,-4,64,110,53,15,30,39,97,48,16,25,25
1,UCLA,20,8,67,94,47,17,24,29,111,50,11,32,29
2,Average,10,2,65,102,50,16,27,34,104,49,13,28,27


Pregame: Lean UCLA at -13.5

Away: Shooting, Rebounding, Free Throw

Home: Turnovers, Rebounding

In [18]:
game_summary('Texas Southern','Kansas')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Texas Southern,-2,-6,69,99,48,18,33,33,93,44,15,27,34
1,Kansas,21,11,69,97,46,16,28,27,111,54,15,33,32
2,Average,9,2,69,98,47,17,31,30,102,49,15,30,33


Pregame: Pass at -21.5

Away: Rebounding, Free Throws

Home: Shooting, Rebounding